In [3]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [4]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

In [5]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [15]:
ya=dict()

def naver_crawl(acode, current):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
    title_result = [] #{acode : {}}
    global ya
    seen_date = current.strftime("%Y.%m.%d %H:%M")

        

    TIME_STOP = (current - datetime.timedelta(minutes = 5)).strftime("%Y.%m.%d %H:%M")


    for page in pages:
     
        pages.append(page + 1)
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(acode,page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')
        for k in tt[2:]:
            if len(k.select('td > span')) > 0:
                date = k.select('td > span')[0].text
                print('hi', date)
                if TIME_STOP >= date or date == seen_date : 
                    print('범위 밖 ㅃㅇ')
                    break


                title = k.select('td.title > a')[0]['title']
                view = k.select('td > span')[1].text
                view = int(view)
               # pos = k.select('td > strong')[0].text
               # neg = k.select('td > strong')[1].text
      

                put_data = (date,title,view)
                              
                                     # 'pos':pos,
                                     #  'neg':neg
                                                 
                print(put_data)
                title_result.append(put_data)
        
        ya[acode] = title_result
        return 
# 1, 코드 ()

In [16]:
result = naver_crawl(ACODE['카카오'], datetime.datetime(2022,1,9,21,20))
# ACODE.keys()

hi 2022.01.09 21:31
('2022.01.09 21:31', '게시판에 공매견들이 ', 2)
hi 2022.01.09 21:31
('2022.01.09 21:31', '50만원 짜리 회사가', 2)
hi 2022.01.09 21:29
('2022.01.09 21:29', '배당금 30원 실화임??', 9)
hi 2022.01.09 21:27
('2022.01.09 21:27', '바닥이 안 보인다', 11)
hi 2022.01.09 21:27
('2022.01.09 21:27', '달러 환율 1200원까지 상승한 원인', 11)
hi 2022.01.09 21:19
('2022.01.09 21:19', '카카오 20만원은 반드시 돌파한다 유태자본 중국자본의 카카오 ', 74)
hi 2022.01.09 21:17
('2022.01.09 21:17', '목표가 5만원', 28)
hi 2022.01.09 21:17
('2022.01.09 21:17', '개잡주네', 23)
hi 2022.01.09 21:09
범위 밖 ㅃㅇ


In [17]:
ya

{'035720': [('2022.01.09 21:31', '게시판에 공매견들이 ', 2),
  ('2022.01.09 21:31', '50만원 짜리 회사가', 2),
  ('2022.01.09 21:29', '배당금 30원 실화임??', 9),
  ('2022.01.09 21:27', '바닥이 안 보인다', 11),
  ('2022.01.09 21:27', '달러 환율 1200원까지 상승한 원인', 11),
  ('2022.01.09 21:19', '카카오 20만원은 반드시 돌파한다 유태자본 중국자본의 카카오 ', 74),
  ('2022.01.09 21:17', '목표가 5만원', 28),
  ('2022.01.09 21:17', '개잡주네', 23)]}

In [23]:
FIVE_MIN = {1:ya}

In [24]:
FIVE_MIN 

{1: {'035720': [('2022.01.09 21:31', '게시판에 공매견들이 ', 2),
   ('2022.01.09 21:31', '50만원 짜리 회사가', 2),
   ('2022.01.09 21:29', '배당금 30원 실화임??', 9),
   ('2022.01.09 21:27', '바닥이 안 보인다', 11),
   ('2022.01.09 21:27', '달러 환율 1200원까지 상승한 원인', 11),
   ('2022.01.09 21:19', '카카오 20만원은 반드시 돌파한다 유태자본 중국자본의 카카오 ', 74),
   ('2022.01.09 21:17', '목표가 5만원', 28),
   ('2022.01.09 21:17', '개잡주네', 23)]}}

In [ ]:
def message():
    start = datetime.datetime.now()
    print("스케쥴 실행중...")
    for key,value in ACODE.items():
        result = naver_crawl(value,datetime.datetime.now())
        
    end = datetime.datetime.now()
    print("실행 : ", end - start)

In [ ]:
import time
import random
import datetime
from kafka import KafkaProducer
#20분 -> 25분까지 25분 30초에 다시 돌기 시작 -> 
bootstrap_servers = ['localhost:9092']  # kafka broker ip
topicName = 'NAVER'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

job = schedule.every(60).seconds.do(message)
while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    if datetime.datetime.now() == NOW + datetime.timedelta(minutes = 1):

        producer.send(topicName, str(job()).encode())

        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        producer.send(topicName, tim.encode())
        break
#5분단위 크롤링 -> 올리고 새로 쓰고 지우고 (이 친구인가,,,) 
#프로듀서 오늘 9시부터 4시까지 긁고 -> 컨슈머에서 긁어와서 쓰기. (실시간....)
#실행 순간 첫 페이지부터, 실시간으로 되려면 생길 때 마다

In [ ]:
import schedule
import time

# step2.실행할 함수 선언
def message():
    print("스케쥴 실행중...")
    for key,value in ACODE.items():
        result = naver_crawl(value,datetime.datetime.now())
        #for i in result:
            #print(i)
#         return

    
# step3.실행 주기 설정
job = schedule.every(60).seconds.do(message)




   


# step4.스캐쥴 시작
count = 0
#current = datetime.datetime.now()
#five_minutes_later = current + datetime.timedelta(minutes=5)




#3시 5분 - 3시 10분 => 15분 예측
# 현재 3시 5분이면, 3시부터- 3시 4분 => 5분이나 10분의 데이터 예측..?
while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    #time.sleep(1)
    
    

   # if datetime.datetime.now().strftime("%H:%M") == NOW.strftime("%H:%M"):
   #     schedule.cancel_job(job)